# Проект: стоимость подержанного автомобиля

## Описание проекта

Многие знают про маркетплейсы где продаются б/у вещи, на которых есть возможность недорого купить качественную и полезную вещь. Но всегда волнует вопрос - кто и как устанавливает цену, и какие его характеристики больше всего влияют на итоговую стоимость продажи?! Вопрос становится особо актуальным, если речь идет про дорогие товары, например про автомобили!

Нам необходимо разработать модель, которая максимально точно сможет предсказать стоимость автомобиля на вторичном рынке

## Описание данных

- Date: Год выпуска автомобиля.
- Make: Марка автомобиля.
- Model: издание автомобиля определенной марки.
- Trim: Уровни отделки салона автомобиля — это просто разные версии модели.
- Body: Тип кузова транспортного средства относится к форме и модели конкретной марки автомобиля.
- Transmission: механизм, который передает мощность от двигателя к колесам.
- VIN: идентификационный номер транспортного средства.
- State: состояние, в котором автомобиль выставлен на аукцион.
- Condition: Состояние автомобилей на момент аукциона.
- Odometer: расстояние, пройденное автомобилем с момента выпуска.
- Color: Цвет кузова автомобиля.
- Interior: Цвет салона автомобиля.
- Seller: Продавец автомобиля, автосалоны.
- mmr: Рекорд рынка Manhiem, рыночная оценочная цена автомобилей.
- sellingprice: цена, по которой автомобиль был продан на аукционе.
- saledate: Дата продажи автомобиля.

# Открываем файлы

## Импортируем библиотеки

In [1]:
import pandas as pd
#from ydata_profiling import ProfileReport
import numpy as np
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from vininfo import Vin
from pyvin import VIN
from tqdm import tqdm
import seaborn as sns

##import warnings
##warnings.filterwarnings("ignore")

In [2]:
#tqdm.pandas()

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
#Объявим константу(ы)
RS = 31415

## Открываем файлы

In [5]:
data_train = pd.read_csv('./datasets/train.csv')
data_test = pd.read_csv('./datasets/test.csv')

Ydata Profiling категорически отказался работать с этим датасетом, в телеграм чате никакого решения не нашли. Другие популярные библиотеки для data profiling'a тоже отказались работать, поэтому придётся изучить данные вручную.

In [6]:
#profile_train = ProfileReport(data_train)
#profile_train
#data_test.profile_report()

## Знакомство с данными

In [7]:
#Используем функцию для ознакомления с данными
def eda(data):
    data.info()
    print()
    data.describe()
    print()
    print(f"Очевидных строк-дубликатов в датасете: {data.duplicated().sum()}")
    print()
    print(f"Пропусков в датасете: {data.isna().sum()}")

In [8]:
data_train.sample(10, random_state=RS)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
8068,2007,Ford,Taurus,SE Fleet,Sedan,automatic,1fafp53u57a105709,az,1.9,135713.0,silver,gray,california auto finance,950,Thu May 21 2015 05:00:00 GMT-0700 (PDT)
85943,2013,Ram,1500,Tradesman,Regular Cab,automatic,3c6jr7at5dg533076,nv,4.4,24692.0,blue,gray,lexus of las vegas,22500,Fri Feb 06 2015 03:45:00 GMT-0800 (PST)
340122,2012,Ram,1500,Sport,Crew Cab,automatic,1c6rd7mt4cs187784,pa,3.4,38455.0,blue,black,randy ford inc,26100,Fri Feb 27 2015 01:15:00 GMT-0800 (PST)
354209,2014,Dodge,Grand Caravan,R/T,Minivan,automatic,2c4rdgeg7er431135,pa,4.0,17539.0,gray,black,enterprise holdings/gdp,18800,Thu Jan 22 2015 01:30:00 GMT-0800 (PST)
390146,2014,Toyota,Corolla,LE,sedan,NaN,2t1burhe2ec060120,pa,4.1,33424.0,gray,gray,kentland car co inc,13000,Fri Jun 05 2015 02:00:00 GMT-0700 (PDT)
354843,2006,Nissan,Altima,2.5,Sedan,automatic,1n4al11d16c189225,nc,2.9,159968.0,silver,black,dt credit corporation,2300,Tue Jan 20 2015 01:30:00 GMT-0800 (PST)
425271,2007,NaN,NaN,NaN,NaN,manual,1g1aj55f577282934,mi,4.5,64205.0,gold,tan,automobiles paille inc,4000,Thu Jun 18 2015 02:30:00 GMT-0700 (PDT)
248415,1999,Ford,Ranger,XLT,Regular Cab,manual,1ftyr10c9xta51711,fl,1.9,225065.0,black,gray,rick case hyundai inc,700,Thu Jan 08 2015 03:00:00 GMT-0800 (PST)
164965,2012,GMC,Acadia,SLT-1,suv,automatic,1gkkrred3cj268915,ca,3.2,16710.0,silver,black,fiserv/usb dealer services northstar exchange,26250,Tue Jun 02 2015 05:30:00 GMT-0700 (PDT)
386412,2014,Toyota,Camry,SE,Sedan,automatic,4t1bf1fk2eu400837,nv,3.7,11431.0,black,black,toyota motor sales usa inc/program,17200,Fri Jan 23 2015 04:00:00 GMT-0800 (PST)


In [9]:
eda(data_train)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          432193 non-null  object 
 2   model         432113 non-null  object 
 3   trim          431899 non-null  object 
 4   body          429843 non-null  object 
 5   transmission  388775 non-null  object 
 6   vin           440236 non-null  object 
 7   state         440236 non-null  object 
 8   condition     430831 non-null  float64
 9   odometer      440167 non-null  float64
 10  color         439650 non-null  object 
 11  interior      439650 non-null  object 
 12  seller        440236 non-null  object 
 13  sellingprice  440236 non-null  int64  
 14  saledate      440236 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 50.4+ MB


Очевидных строк-дубликатов в датасете: 0

Пропусков в датасете: year       

In [10]:
data_test.sample(10, random_state=RS)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,saledate
67501,2010,Ford,F-250 Super Duty,XLT,crew cab,automatic,1ftsw2by1aea15987,mi,2.9,74331.0,green,gray,automobiles paille inc,Thu Jun 11 2015 02:30:00 GMT-0700 (PDT)
62053,2012,Toyota,Tundra,Tundra FFV,CrewMax Cab,automatic,5tfdw5f17cx213066,oh,3.6,31822.0,white,gray,wells fargo dealer services,Tue Jan 13 2015 12:00:00 GMT-0800 (PST)
62846,2013,Ford,Explorer,XLT,SUV,automatic,1fm5k7d85dgc29016,tn,4.7,23331.0,—,black,"ford motor credit company,llc",Thu Feb 19 2015 03:00:00 GMT-0800 (PST)
32852,2011,Nissan,Armada,Platinum,SUV,automatic,5n1aa0ne2bn621697,pa,4.5,30462.0,gray,black,nissan-infiniti lt,Fri Jan 16 2015 01:00:00 GMT-0800 (PST)
78482,2012,Ford,Taurus,SEL,Sedan,automatic,1fahp2ew4cg129450,fl,4.7,28882.0,—,gray,"ford motor credit company,llc",Wed Jan 07 2015 10:00:00 GMT-0800 (PST)
58063,2012,Nissan,Rogue,S,suv,automatic,jn8as5mv4cw354157,fl,4.4,21389.0,white,gray,nissan-infiniti lt,Tue Jun 16 2015 02:30:00 GMT-0700 (PDT)
110025,2008,Chevrolet,Malibu,Fleet,sedan,automatic,1g1zg57n98f189637,il,2.2,135725.0,silver,gray,nationwide acceptance,Thu May 28 2015 07:00:00 GMT-0700 (PDT)
62442,2011,Nissan,Rogue,SV,SUV,automatic,jn8as5mv6bw311857,fl,4.6,35492.0,gray,black,nissan-infiniti lt,Wed Feb 04 2015 01:00:00 GMT-0800 (PST)
60727,1996,BMW,Z3,1.9,Convertible,manual,4usch7327tlb67564,ga,3.0,35782.0,red,black,wells fargo dealer services,Thu Mar 05 2015 02:00:00 GMT-0800 (PST)
55435,2009,Nissan,Rogue,SL,SUV,automatic,jn8as58t79w327035,fl,2.5,125880.0,gray,gray,westfall auto sales,Tue Feb 03 2015 08:00:00 GMT-0800 (PST)


In [11]:
eda(data_test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110058 entries, 0 to 110057
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          110058 non-null  int64  
 1   make          107997 non-null  object 
 2   model         107979 non-null  object 
 3   trim          107944 non-null  object 
 4   body          107464 non-null  object 
 5   transmission  97047 non-null   object 
 6   vin           110058 non-null  object 
 7   state         110058 non-null  object 
 8   condition     107679 non-null  float64
 9   odometer      110039 non-null  float64
 10  color         109900 non-null  object 
 11  interior      109900 non-null  object 
 12  seller        110058 non-null  object 
 13  saledate      110058 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 11.8+ MB


Очевидных строк-дубликатов в датасете: 0

Пропусков в датасете: year                0
make             2061
model      

**Выводы:**
<br>Сразу видим достаточное большое количество пропусков в данных. Также в некоторых столбцах нужно будет изменить тип данных, чтобы либо вытащить данные и создать новые признаки, либо просто оптимизировать. Также нужно будет проверить данные на очевидные и неочевидные дубликаты, которых, вероятнее всего, окажется немало.

# Предобработка данных

## Дубликаты

Для начала поработаем с регистром в категориальных столбцах, чтобы избавиться от неочевидных дубликатов. Также избавимся от лишних пробелов. Будем обрабатывать сразу оба датасета

In [12]:
ds_list = [data_train, data_test]

### Дубликаты регистра и лишних пробелов

После первого прогона удалил те столбцы, где операция не дала никаких изменений

In [13]:
for data in ds_list:
    for column in ['make', 'model', 'trim', 'body']:
        print(f"Уникальных значений в столбце {column} ДО обработки: {data[column].nunique()}")
        data[column] = data[column].str.strip()
        data[column] = data[column].str.lower()
        print(f"Уникальных значений в столбце {column} ПОСЛЕ обработки: {data[column].nunique()}")
        print()
        print()

Уникальных значений в столбце make ДО обработки: 92
Уникальных значений в столбце make ПОСЛЕ обработки: 62


Уникальных значений в столбце model ДО обработки: 959
Уникальных значений в столбце model ПОСЛЕ обработки: 839


Уникальных значений в столбце trim ДО обработки: 1922
Уникальных значений в столбце trim ПОСЛЕ обработки: 1840


Уникальных значений в столбце body ДО обработки: 85
Уникальных значений в столбце body ПОСЛЕ обработки: 45


Уникальных значений в столбце make ДО обработки: 85
Уникальных значений в столбце make ПОСЛЕ обработки: 58


Уникальных значений в столбце model ДО обработки: 824
Уникальных значений в столбце model ПОСЛЕ обработки: 746


Уникальных значений в столбце trim ДО обработки: 1496
Уникальных значений в столбце trim ПОСЛЕ обработки: 1448


Уникальных значений в столбце body ДО обработки: 77
Уникальных значений в столбце body ПОСЛЕ обработки: 42




### Неочевидные дубликаты в body

In [14]:
data_train['body'].sort_values().unique()

array(['access cab', 'beetle convertible', 'cab plus', 'cab plus 4',
       'club cab', 'convertible', 'coupe', 'crew cab', 'crewmax cab',
       'cts coupe', 'cts wagon', 'cts-v coupe', 'cts-v wagon',
       'double cab', 'e-series van', 'elantra coupe', 'extended cab',
       'g convertible', 'g coupe', 'g sedan', 'g37 convertible',
       'g37 coupe', 'genesis coupe', 'granturismo convertible',
       'hatchback', 'king cab', 'koup', 'mega cab', 'minivan',
       'promaster cargo van', 'q60 convertible', 'q60 coupe', 'quad cab',
       'ram van', 'regular cab', 'regular-cab', 'sedan', 'supercab',
       'supercrew', 'suv', 'transit van', 'tsx sport wagon', 'van',
       'wagon', 'xtracab', nan], dtype=object)

Тут не находим ничего интересного, заменим 1 лишнее значение и пойдём дальше

In [15]:
data_train['body'].replace('koup', 'coupe', inplace=True)
data_test['body'].replace('koup', 'coupe', inplace=True)

## Пропуски

In [16]:
data_train.isna().sum()

year                0
make             8043
model            8123
trim             8337
body            10393
transmission    51461
vin                 0
state               0
condition        9405
odometer           69
color             586
interior          586
seller              0
sellingprice        0
saledate            0
dtype: int64

Видим 

Пропуски в color не так страшны, поскольку вряд ли влияют на цену хоть как-то значительно, а с transmission придётся исправлять ситуацию.

### Пропуски в color

In [17]:
nan_color_train = len(data_train.query('color == "—"'))
nan_color_test = len(data_test.query('color == "—"'))

print(f"Всего пропусков в color в data_test {nan_color_test + data_test['color'].isna().sum()}")
print()
f"Всего пропусков в color в data_train {nan_color_train + data_train['color'].isna().sum()}"

Всего пропусков в color в data_test 5106



'Всего пропусков в color в data_train 20106'

In [18]:
#Замена -
data_train['color'] = data_train['color'].replace('—', 'unknown')
data_test['color'] = data_test['color'].replace('—', 'unknown')
#Заполнение пустых ячеек
data_train['color'].fillna('unknown', inplace=True)
data_test['color'].fillna('unknown', inplace=True)

In [19]:
f"Пропусков в train:{data_train['color'].isna().sum()}, в test: {data_test['color'].isna().sum()}"

'Пропусков в train:0, в test: 0'

### Пропуски в make

Поскольку библиотека вносит очень много новых вариаций брендов, часть с обработкой дубликатов в столбце перенесём сюда

**Data_train**

In [20]:
data_train.loc[data_train['make'].isna(), 'make'] = data_train.loc[data_train['make'].isna(), 'vin'].map(lambda x: Vin(x).manufacturer)
data_train['make'] = data_train.make.str.lower()
print(f"Пропусков в data_train, столбце make после обработки: {data_train['make'].isna().sum()}")

Пропусков в data_train, столбце make после обработки: 0


**Data_ test**

In [21]:
data_test.loc[data_test['make'].isna(), 'make'] = data_test.loc[data_test['make'].isna(), 'vin'].map(lambda x: Vin(x).manufacturer)
data_test['make'] = data_test.make.str.lower()
print(f"Пропусков в data_test, столбце make после обработки: {data_test['make'].isna().sum()}")

Пропусков в data_test, столбце make после обработки: 0


### Дубликаты в make

Создадим словарь, с помощью которого будем заменять неявыные дубликаты

In [22]:
make_replacements = {
    'am' : 'aston martin',
    'bmw m' : 'bmw',
    'chevrolet canada' : 'chevrolet',
    'chevrolet mexico' : 'chevrolet',
    'chevrolet usa' : 'chevrolet',
    'chrysler canada' : 'chrysler',
    'dodge tk' : 'dodge',
    'dodge canada' : 'dodge',
    'dodge mexico' : 'dodge',
    'ford truck' : 'ford',
    'gmc truck' : 'gmc',
    'ford tk' : 'ford',
    'hyundai tk' : 'hyundai',
    'landrover' : 'land rover',
    'mazda tk' : 'mazda',
    'mercedes-b' : 'mercedes-benz',
    'mercedes' : 'mercedes-benz',
    'mercedes-benz (sprinter)' : 'mercedes-benz',
    'porsche car' : 'porsche',
    'porsche suv' : 'porsche',
    'vw' : 'volkswagen',
    'volkswagen commercial vehicles' : 'volkswagen'
}

for data in ds_list:
    data['make'].replace(make_replacements, inplace=True)
    
#Проверяем изменения
#data_test['make'].sort_values().unique()
#data_train['make'].sort_values().unique()

### Пропуски в model

Пропуски будем заполнять с помощью библиотеки pyvin. Поскольку работает она крайне медленно (3 итерации в секунду в лучшем случае), то результат подбора буду сохранять в файл и перезаписывать переменные, а процесс подбора закомментирую

In [23]:
#Создадим функцию для получения модели авто из vin ( Anatoly Ashulin <3 )
def model_parsing(x):
    try:
        return VIN(x).Model
    except:
        return np.nan

**Data train**

In [24]:
#data_train['vin'] = data_train.vin.str.upper()
#data_train.loc[data_train['model'].isna(), 'model'] = data_train.loc[data_train['model'].isna(), 'vin'].progress_apply(model_parsing)
#data_train.to_csv(r'C:\Users\kotof\Documents\YandexPracticum\workshop-one\datasets\data_train_preprocessed_v1')

In [25]:
#Перезапишем data_train вариантом с уже заполненными пропусками
data_train = pd.read_csv('./datasets/data_train_preprocessed_v1.csv')

Остаётся 57 пропусков, которые мы спокойно можем заполнить.

In [26]:
data_train['model'].fillna('unknown', inplace=True)

**Data test**

In [27]:
#data_test['vin'] = data_test.vin.str.upper()
#data_test.loc[data_test['model'].isna(), 'model'] = data_test.loc[data_test['model'].isna(), 'vin'].progress_apply(model_parsing)
#data_test.to_csv(r'C:\Users\kotof\Docum ents\YandexPracticum\workshop-one\datasets\data_test_preprocessed_v1.csv')

Аналогично заполняем оставшиеся пропуски (15 штук). Все пропуски model обработаны

In [28]:
data_test = pd.read_csv('./datasets/data_test_preprocessed_v1.csv')

In [29]:
data_test['model'].fillna('unknown', inplace=True)

Забыл указать index=false в to_csv, поэтому удалим лишние столбцы

In [30]:
data_train.drop(columns=['Unnamed: 0'], inplace=True)
data_test.drop(columns=['Unnamed: 0'], inplace=True)

In [31]:
data_train.drop(columns=['manufacturer_vininfo'], inplace=True)

### Пропуски в interior

Тут пропусков крайне мало, заполняем их unknown

In [32]:
data_train['interior'].fillna('unknown', inplace=True)
data_test['interior'].fillna('unknown', inplace=True)

### Пропуски в odometer

Опять таки, пропусков крайне мало, заполним их медианным значением

In [33]:
data_train.loc[data_train['odometer'].isna(), 'odometer'] = data_train['odometer'].median()
data_test.loc[data_test['odometer'].isna(), 'odometer'] = data_test['odometer'].median()

### Пропуски в trim, body и condition

К сожалению, из vin информацию по этим признакам не получить, а другого способа, кроме как создавать отдельную модель, у нас нет, поэтому заполним их unknown.

In [34]:
data_train['trim'].fillna('unknown', inplace=True)
data_test['trim'].fillna('unknown', inplace=True)

data_train['body'].fillna('unknown', inplace=True)
data_test['body'].fillna('unknown', inplace=True)

data_train['condition'].fillna('unknown', inplace=True)
data_test['condition'].fillna('unknown', inplace=True)

### Пропуски в transmission

Остаётся самый проблемный признак - коробка передач. В нём мы видим более 10% пропусков в обучающем дс, примерно столько же в тестовом. Попробуем найти авто, для которых есть только два варианта записей:
manual\automatic и nan. Так мы отсеем те модели, которые выпускаются только с одной коробкой передач.

In [35]:
#%%time
#only_automatic_train = []
#only_manual_train = []
#
#only_automatic_test = []
#only_manual_test = []
#
#for data in ds_list:
#    for model in data.loc[data['transmission'].isna(), 'model'].unique():
#        unique_trs = data.loc[data['model'] == model, 'transmission'].unique()
#        if len(unique_trs) == 2:
#            if len(data) > 400000:
#                if 'automatic' in unique_trs:
#                    only_automatic_train.append(model)
#                elif 'manual' in unique_trs:
#                    only_manual_train.append(model)
#            elif len(data) < 150000:
#                if 'automatic' in unique_trs:
#                    only_automatic_test.append(model)
#                elif 'manual' in unique_trs:
#                    only_manual_test.append(model)
#            
#

In [36]:
##Data train
#print(f"Пропусков до обработки: {data_train['transmission'].isna().sum()}")
#automatic_indexes_train = data_train.query('transmission.isna() and model in @only_automatic_train').index
#data_train.loc[automatic_indexes_train, 'transmission'] = 'automatic'
#
#manual_indexes_train = data_train.query('transmission.isna() and model in @only_manual_train').index
#data_train.loc[manual_indexes_train, 'transmission'] = 'manual'
#print(f"Пропусков после обработки: {data_train['transmission'].isna().sum()}")

In [37]:
##Data test
#print(f"Пропусков до обработки: {data_test['transmission'].isna().sum()}")
#automatic_indexes_test = data_test.query('transmission.isna() and model in @only_automatic_test').index
#data_test.loc[automatic_indexes_test, 'transmission'] = 'automatic'
#
#manual_indexes_test = data_test.query('transmission.isna() and model in @only_manual_test').index
#data_test.loc[manual_indexes_test, 'transmission'] = 'manual'
#print(f"Пропусков после обработки: {data_test['transmission'].isna().sum()}")

**Итоги:**
<br>Итого получилось удалить по 30% пропусков. Остальное заполним unknown. Поскольку выполнение цикла занимает 15 секунд, сохраним результат в csv, и перезапишем переменные.

In [38]:
#data_train.to_csv(r'C:\Users\kotof\Documents\YandexPracticum\workshop-one\datasets\data_train_preprocessed_v2.csv', index=False)
#data_test.to_csv(r'C:\Users\kotof\Documents\YandexPracticum\workshop-one\datasets\data_test_preprocessed_v2.csv', index=False)#

In [39]:
data_train = pd.read_csv('./datasets/data_train_preprocessed_v2.csv')
data_test = pd.read_csv('./datasets/data_test_preprocessed_v2.csv')

In [40]:
data_train['transmission'].fillna('unknown', inplace=True)
data_test['transmission'].fillna('unknown', inplace=True)

## Тип данных

### saledate

In [41]:
#Data train
data_train['saledate'] = data_train['saledate'].apply(lambda x: x[:15])
data_train['saledate'] = pd.to_datetime(data_train['saledate'], format='%a %b %d %Y')
#Data test
data_test['saledate'] = data_test['saledate'].apply(lambda x: x[:15])
data_test['saledate'] = pd.to_datetime(data_test['saledate'], format='%a %b %d %Y')

In [42]:
data_train.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
0,2011,ford,edge,sel,suv,automatic,2FMDK3JC4BBA41556,md,4.2,111041.0,black,black,santander consumer,12500,2015-06-02
1,2014,ford,fusion,se,sedan,automatic,3FA6P0H75ER208976,mo,3.5,31034.0,black,black,ars/avis budget group,14500,2015-02-25
2,2012,nissan,sentra,2.0 sl,sedan,automatic,3N1AB6AP4CL698412,nj,2.2,35619.0,black,black,nissan-infiniti lt,9100,2015-06-10
3,2003,hummer,h2,base,suv,automatic,5GRGN23U93H101360,tx,2.8,131301.0,gold,beige,wichita falls ford lin inc,13300,2015-06-17
4,2007,ford,fusion,sel,sedan,automatic,3FAHP08Z17R268380,md,2.0,127709.0,black,black,purple heart,1300,2015-02-03


### odometer

In [43]:
data_train['odometer'] = data_train['odometer'].astype('int32')
data_test['odometer'] = data_test['odometer'].astype('int32')

### selling_price

In [44]:
data_train['sellingprice'] = data_train['sellingprice'].astype('int32')

## Переименование столбцов

In [45]:
%%capture
data_train.rename(columns={"make": "manufacturer", "sellingprice": "selling_price", "saledate": "sale_date", "year" :"manufacturing_year"}, inplace=True)
data_test.rename(columns={"make": "manufacturer", "saledate": "sale_date", "year" :"manufacturing_year"}, inplace=True)

In [46]:
data_train.head()

,manufacturing_year,manufacturer,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,selling_price,sale_date
0,2011,ford,edge,sel,suv,automatic,2FMDK3JC4BBA41556,md,4.2,111041,black,black,santander consumer,12500,2015-06-02
1,2014,ford,fusion,se,sedan,automatic,3FA6P0H75ER208976,mo,3.5,31034,black,black,ars/avis budget group,14500,2015-02-25
2,2012,nissan,sentra,2.0 sl,sedan,automatic,3N1AB6AP4CL698412,nj,2.2,35619,black,black,nissan-infiniti lt,9100,2015-06-10
3,2003,hummer,h2,base,suv,automatic,5GRGN23U93H101360,tx,2.8,131301,gold,beige,wichita falls ford lin inc,13300,2015-06-17
4,2007,ford,fusion,sel,sedan,automatic,3FAHP08Z17R268380,md,2.0,127709,black,black,purple heart,1300,2015-02-03


## Создание новых столбцов

In [47]:
data_train['sale_year'] = pd.DatetimeIndex(data_train['sale_date']).year
data_train['sale_month'] = pd.DatetimeIndex(data_train['sale_date']).month
data_train['age'] = data_train['sale_year'] - data_train['manufacturing_year']

data_test['sale_year'] = pd.DatetimeIndex(data_test['sale_date']).year
data_test['sale_month'] = pd.DatetimeIndex(data_test['sale_date']).month
data_test['age'] = data_test['sale_year'] - data_test['manufacturing_year']

In [48]:
data_train.head()

,manufacturing_year,manufacturer,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,selling_price,sale_date,sale_year,sale_month,age
0,2011,ford,edge,sel,suv,automatic,2FMDK3JC4BBA41556,md,4.2,111041,black,black,santander consumer,12500,2015-06-02,2015,6,4
1,2014,ford,fusion,se,sedan,automatic,3FA6P0H75ER208976,mo,3.5,31034,black,black,ars/avis budget group,14500,2015-02-25,2015,2,1
2,2012,nissan,sentra,2.0 sl,sedan,automatic,3N1AB6AP4CL698412,nj,2.2,35619,black,black,nissan-infiniti lt,9100,2015-06-10,2015,6,3
3,2003,hummer,h2,base,suv,automatic,5GRGN23U93H101360,tx,2.8,131301,gold,beige,wichita falls ford lin inc,13300,2015-06-17,2015,6,12
4,2007,ford,fusion,sel,sedan,automatic,3FAHP08Z17R268380,md,2.0,127709,black,black,purple heart,1300,2015-02-03,2015,2,8


In [49]:
data_test.head()

,manufacturing_year,manufacturer,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sale_date,sale_year,sale_month,age
0,2005,cadillac,cts,base,sedan,automatic,1G6DP567450124779,ca,2.7,116970,silver,black,lexus of stevens creek,2015-01-14,2015,1,10
1,2014,gmc,savana cargo,2500,van,automatic,1GTW7FCA7E1902207,pa,4.4,6286,white,gray,u-haul,2015-02-27,2015,2,1
2,2013,nissan,murano,s,suv,automatic,JN8AZ1MW6DW303497,oh,4.6,11831,gray,black,nissan-infiniti lt,2015-02-24,2015,2,2
3,2013,chevrolet,impala,ls fleet,sedan,automatic,2G1WF5E34D1160703,fl,2.3,57105,silver,black,onemain rem/auto club of miami inc dba north dad,2015-03-06,2015,3,2
4,2013,nissan,titan,sv,crew cab,automatic,1N6AA0EC3DN301209,tn,2.9,31083,black,black,nissan north america inc.,2015-06-03,2015,6,2


## Удалим лишние столбцы

По причине неинформативности или дублирования информативности для модели, удалим лишние столбцы из датасетов

In [50]:
data_train.drop(columns=[
    'vin', #неинформативен
    'sale_date', #дублирует данные за отдельно год и месяц
    'manufacturing_year' #можно восстановить из sale_year и age
], inplace=True)

In [51]:
data_test.drop(columns=[
    #'vin',
    'sale_date',
    'manufacturing_year'
], inplace=True)

## Выбросы


Пройдёмся по количественным признакам, и удалим выбросы


In [52]:
#Функция для удаления выбросов
def remove_outliers(data, column):
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    iqr = q3-q1
    upper = data.loc[data[column] >= (q3 + 1.5 * iqr)].index
    lower = data.loc[data[column] <= (q1 - 1.5 * iqr)].index
    data.drop(upper, inplace=True)
    data.drop(lower, inplace=True)

In [53]:
for data in ds_list:
    print(f"Строк в датасете до удаления выбросов: {data.shape[0]}")
    count = 0
    while count != data.shape[0]:
        count = data.shape[0]
        remove_outliers(data, 'odometer')
    print(f"Строк в датасете после удаления выбросов: {data.shape[0]}")
    print()

Строк в датасете до удаления выбросов: 440236
Строк в датасете после удаления выбросов: 429710

Строк в датасете до удаления выбросов: 110058
Строк в датасете после удаления выбросов: 107432



# Разработка модели

## Выделяем целевой признак и фичи

In [54]:
#Обучающие данные
features_train = data_train.drop(columns=['selling_price'])
target_train = data_train['selling_price']

#Тестовые данные
features_test = data_test.drop(columns=['vin'])

## Масштабируем и кодируем данные

Учитывая количество уникальных значений в наших столбцах, ohe не подойдёт категорически. Вторым вариантом, приходящим на ум, является OrdinalEncoder, но модель может прочитать в его работе зависимости и закономерности, которых на самом деле нет. Например, порядковая модель 47 будет "весить" больше порядковой модели 19, хотя фактически они должны быть равнозначными для модели при прочих равных. Потому, дабы соблюсти баланс между размером датасета и неприданием ложного смысла для модели, выбираем BinaryEncoder

In [55]:
numeric = [*features_train.select_dtypes(include=['int64', 'int32']).columns]
scaler = StandardScaler().fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

In [56]:
be = ce.BinaryEncoder(handle_unknown='use_encoded_value').fit(features_train)

features_train = be.transform(features_train)
features_test = be.transform(features_test)

In [57]:
#Итого:
features_train.head()

,manufacturer_0,manufacturer_1,manufacturer_2,manufacturer_3,manufacturer_4,manufacturer_5,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_9,model_10,trim_0,trim_1,trim_2,trim_3,trim_4,trim_5,trim_6,trim_7,trim_8,trim_9,trim_10,body_0,body_1,body_2,body_3,body_4,body_5,transmission_0,transmission_1,state_0,state_1,state_2,state_3,state_4,state_5,condition_0,condition_1,condition_2,condition_3,condition_4,condition_5,odometer,color_0,color_1,color_2,color_3,color_4,interior_0,interior_1,interior_2,interior_3,interior_4,seller_0,seller_1,seller_2,seller_3,seller_4,seller_5,seller_6,seller_7,seller_8,seller_9,seller_10,seller_11,seller_12,seller_13,sale_year,sale_month,age
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0.797543,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.328663,0.672419,-0.218090
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,-0.696844,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.328663,-0.560021,-0.976742
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,-0.611205,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.328663,0.672419,-0.470974
3,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1.175963,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.328663,0.672419,1.804982
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,1.108871,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0.328663,-0.560021,0.793446


## Выбор модели

В соревновании выбранной метрикой является MAPE, поэтому в GSCV будем опираться на неё.

### CatBoostRegressor

На 4090 подбор занял 45 минут, поэтому лучшие гиперпараметры использую в gridsearchcv, а те, на которых проводился подбор оставлю для ознакомления

In [58]:
#model = CatBoostRegressor(task_type="GPU", devices='0:1', verbose=False, random_state=RS)
#
#parameters_cb = {'depth' : range(5, 20, 2),
#              'learning_rate' : [0.01, 0.05, 0.1, 0.2],
#              'iterations' : range(100, 501, 100)}
##Добавлено уже после работы gscv
#best_params_cb = {'depth': [11], 'iterations': [500], 'learning_rate': [0.1]}
#
#grid_cb = GridSearchCV(estimator=model, param_grid=best_params_cb, cv=5, scoring='neg_mean_absolute_percentage_error')
#cb = grid_cb.fit(features_train, target_train)

In [59]:
#print(f'Лучший набор гиперпараметров: для CatBoost {cb.best_params_}')
#print(f"Лучший MAPE на обучающей выборке: {-(cb.best_score_.round(5))}")

Получаем достаточно хороший результат 0.188. Посмотрим, как себя покажет GBR

### GradientBoostingRegressor

**Обучение модели занимает крайне долго даже с уже готовыми параметрами. Поскольку она показала неудовлетворяющие метрики, закоментирую её**

In [60]:
#model = GradientBoostingRegressor(random_state=RS, verbose = 10)
#
#parameters_gbr = {'loss' : ['squared_error', 'huber', 'quantile', 'absolute_error'],
#    'learning_rate' : [0.1],
#    'n_estimators' : range(100, 601, 100)}
##Добавлено уже после работы gscv
#best_params_gbr = {'learning_rate': [0.1], 'loss': ['absolute_error'], 'n_estimators': [600]}
#
#grid_gbr = GridSearchCV(estimator=model, param_grid=best_params_gbr, cv=5, scoring='neg_mean_absolute_percentage_error', n_jobs= -1)
#gbr = grid_gbr.fit(features_train, target_train)

In [61]:
#print(f'Лучший набор гиперпараметров: для GBR {gbr.best_params_}')
#print(f"Лучший MAPE на обучающей выборке: {-(gbr.best_score_.round(5))}")

Видим результат вдвое хуже, чем с catboost, поэтому gbr отметается. Рассмотрим другие модели

### RandomForestRegressor

In [62]:
#model = RandomForestRegressor(random_state=RS, verbose = 10)
#
#parameters_rfr = {'criterion' : ['squared_error', 'poisson', 'friedman_mse', 'absolute_error'],
#    'max_depth' : [5, 10, 15],
#    'n_estimators' : range(100, 601, 100)}
##Добавлено уже после работы gscv
#best_params_rfr = []
#
#grid_rfr = GridSearchCV(estimator=model, param_grid=parameters_rfr, cv=5, scoring='neg_mean_absolute_percentage_error', n_jobs= -1)
#rfr = grid_rfr.fit(features_train, target_train)

In [63]:
#print(f'Лучший набор гиперпараметров: для RFR {rfr.best_params_}')
#print(f"Лучший MAPE на обучающей выборке: {-(rfr.best_score_.round(5))}")

Получили mape, равный 0.27, что ощутимо хуже результата catboost, поэтому останавливаемся на нём. Ячейки по тем же причинам закоммнетировал.

 Итого, лучшей моделью оказалась CatBoostRegressor с MAPE равным 0.184. Осталось сохранить предсказания, оформить это всё в готовый файл и загрузить на Kaggle

## Фиксируем результат

**Финальная модель**

In [65]:
best_model = CatBoostRegressor(depth=11, iterations=500, learning_rate=0.1, random_state=RS, verbose=False)
best_model.fit(features_train, target_train)

#Сохраняем предсказания и вины в отдельные переменные
final_predictions = best_model.predict(features_test)
vins = pd.read_csv('./datasets/test.csv')['vin']

#Объединяем
kaggle_final = pd.merge(vins, pd.DataFrame(final_predictions), left_index=True, right_index=True)

110058


,vin,0
0,1g6dp567450124779,4631.824073
1,1gtw7fca7e1902207,22277.686542
2,jn8az1mw6dw303497,19328.331204
3,2g1wf5e34d1160703,8920.284065
4,1n6aa0ec3dn301209,22356.171045


In [78]:
kaggle_final.columns= ['vin', 'sellingprice']

In [80]:
#Сохраняем файл
#kaggle_final.to_csv(r'C:\Users\kotof\Documents\YandexPracticum\workshop-one\datasets\kaggle_submission_efkas.csv', index=False)

# Выводы

В этом проекте нашей задачей было разработать модель с наименьшим MAPE для предсказания стоимости подержанных автомобилей на рынке.

Для этих целей были импортированы библиотеки для анализа данных, предобработки и восстановления информации. Была проведена предобработка данных. В ходе неё были заполнены пропуски в столбцах: color, make, model, interior, odometer и transmission. Благодаря библиотекам pyvin и vininfo удалось заполнить пропуски о моделях и производителях без каких-либо компромиссов и заглушек. Также была создана функция, чтобы выявить модели авто только с автоматической и механической коробкой передач, и были заполнены 30% пропусков в transmission таким образом. 

Также были обработаны различные дубликаты. В первую очередь все текстовые значения были приведены к нижнему регистру, в них были убраны лишние пробелы. Таким образом уже удалось устранить значительную часть неочевидных дубликатов. После этого были изучены столбцы по отдельности и их уникальные значения (где возможно), и были устранены дополнительные дубликаты. Были исправлены названия и дубли автомобилей в столбце make.

Также были изменены типы данных. Последний столбец был приведён к нужному виду, после чего его удалось перевести в datetime, числовые признаки были переведены в тип int32 ввиду ненадобности 64 бит для подобных значений. Благодаря переводу столбца saledate в datetime удалось создать новые столбцы с отдельными годами и месяцами продажи, а также возрастом автомобилей, а от изначального избавиться.Также были удалены выбросы методом межквартильного размаха.

Перед разработкой модели данные были масштабированы и кодированы. Если с масштабированием никаких вопрос не возникает, то привычный нам из OHE не подходил, поскольку в датасете мы имеем десятки тысяч уникальных значений. Вторым вариантом был OrdinalEncoder, который тоже не очень хорошо подходил на эту роль, поскольку вносил в данные зависимости, которых на самом деле нет. К примеру, порядковая модель 50 будет "весить" для модели больше, чем порядковая модель 15, хотя в действительности такого соотношения в значимости нет. По этой причине был выбран BinaryEncoder. Он позволял нам сохранить адекватный размер обработанного датафрейма, а так же не вносить новую некорректную информацию в данные.

После этого были протестированы несколько моделей. GradientBoostRegressor, CatBoostRegressor и RandomForestRegressor. Хуже всего себя показал GBR, выдав MAPE равный 35%. Первая же модель, а именно CBR, показала наилучший MAPE, равный 18%, по этой причине выбор пал на неё. Плюс она относительно быстрее своих собратьев.

Файл с вин номерами и предсказаниями был загружен на kaggle, до окончания соревнования MAPE на тестовых данных составил 15%, что является достаточно хорошим результатом.

**Спасибо всем студентам практикума, кто также работал над этим проектом, и помогал в общем чате при необходимости. Рад был работать вместе!**